In [1]:
import pandas as pd
import numpy as np
import json as js
from datetime import datetime
from dateutil.relativedelta import relativedelta

#read in csv and/or dataset
water_df = pd.read_csv('../SourceMaterial/water_temps.csv')
water_df = water_df.dropna()
#order by HUC then Date
water_df.sort_values(by=['HUCNAME_', 'Date'])
# water_df.set_index('HUCNAME_')
# variables = water_df.columns().drop('HUCNAME_')
hucs = water_df['HUCNAME_'].unique()
results = []

# def datecheck(string):
#     df = water_df.loc[water_df['HUCNAME_']==string]
#     df['datetime'] = [datetime.strptime(value, '%m/%d/%Y') for value in df['Date']]
#     df['datediff'] = df['datetime'].diff
#     datedifcol = df['datediff']
#     datetimecol = df['datetime']
#     max_diff = max(datedifcol)
#     max_date = max(datetimecol)
#     min_date = min(datetimecol)
#     print(max_diff, max_date, min_date)
#     diff_years = relativedelta(max_date, min_date).years
#     if diff_years >=3 & max_diff <= 7:
#         return True
#     else:
#         return False
                                    


# for each HUC, check if dates are continuous over at least 3 years, build list of suitable HUCs...check

#for each suitable HUC, split temps into quarters by date...check

#calculate variance or stdev of temps and avg temps for each quarter

#for each quarter calculate increase in avg over time

#return suitable hucs by order of highest degree of change, and greatest variability

C:\Users\jacob\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
water_df

,Date,ModifiedDate,HUCNAME_,Latitude,Longitude,Parameter,ParameterName_CBP,ParameterName_CMC,MeasureValue
0,01/01/2012,2020-04-15T14:35:09.607,Hazel Run-Rappahannock River,38.289800,-77.481600,WT.8,WATER TEMPERATURE DEG,Water temperature,9.0
1,01/01/2014,2020-03-10T11:04:49.987,Totuskey Creek,37.923634,-76.720680,WT.8,WATER TEMPERATURE DEG,Water temperature,6.0
2,01/01/2014,2020-03-10T11:16:13.723,Totuskey Creek,37.886036,-76.736281,WT.8,WATER TEMPERATURE DEG,Water temperature,6.0
3,01/01/2014,2020-03-10T11:04:35.813,Totuskey Creek,37.926034,-76.709080,WT.8,WATER TEMPERATURE DEG,Water temperature,6.0
4,01/01/2014,2020-03-10T11:04:18.423,Totuskey Creek,37.912934,-76.657379,WT.8,WATER TEMPERATURE DEG,Water temperature,5.0
...,...,...,...,...,...,...,...,...,...
42068,12/30/2005,2020-03-27T10:35:46.52,West Branch Little Conestoga Creek-Little Cone...,39.983782,-76.377848,WT.13,WATER TEMPERATURE DEG,Water Temperature,8.0
42069,12/30/2010,2020-03-27T10:36:03.91,Lititz Run,40.133611,-76.258056,WT.13,WATER TEMPERATURE DEG,Water Temperature,7.0
42070,12/30/2013,2020-03-27T10:36:03.723,Lititz Run,40.133611,-76.258056,WT.13,WATER TEMPERATURE DEG,Water Temperature,7.5
42071,12/31/2004,2020-03-26T12:02:27.74,Little Cocalico Creek-Cocalico Creek,40.187778,-76.170000,WT.13,WATER TEMPERATURE DEG,Water Temperature,4.0


In [3]:
for huc in hucs:
       
    
    #checks dates for continuity and returns datecheck as boolean
    df = water_df.loc[water_df['HUCNAME_']==huc]
    df['datetime'] = [datetime.strptime(value, '%m/%d/%Y') for value in df['Date']]
    df = df.sort_values(by=['datetime'])
    df['datediff'] = df['datetime'].diff()
    datetimecol = df['datetime']
    min_date = min(datetimecol)
    max_date = max(datetimecol)
    diff_years = relativedelta(max_date, min_date).years
    df = df.dropna()
    datedifcol = df['datediff'].dt.days
    try:
        max_diff = max(datedifcol)
        if diff_years >=3 and max_diff <= 100:
            datecheck = True
        else:
            datecheck = False
    except:
        datecheck = False
    if datecheck == True:
#         results.append()
        #for each suitable HUC, split temps into quarters by date
        df['month'] = [int(value[0:1]) for value in df['Date']]
        df['year'] = [value[6:10] for value in df['Date']]
        df['q'] = ''
        
        df['q'].loc[df['month'] <=3] = 'Q1'
        df['q'].loc[df['month'] >=4] = 'Q2'
        df['q'].loc[df['month'] >=7] = 'Q3'
        df['q'].loc[df['month'] >=10] = 'Q4'
        
        df['Quarter'] = df['q'] + " "+ df['year']
        
        df.drop(columns = ['month', 'year'])
        
        #calculate variance or stdev of temps and avg temps for each quarter
        quarters_list = df['Quarter'].unique()    
        for quarter in quarters_list:
            quarter_df = df.loc[df['Quarter']==quarter]
            values = [value for value in quarter_df['MeasureValue']]
            variance = np.var(values)
            stdev = np.std(values)
            average = np.average(values)
            max_val = max(values)
            min_val = min(values)
            results.append(
                {
                    'huc':huc, 
                    'quarter': quarter, 
                    'var' : variance, 
                    'stdev' : stdev, 
                    'avg' : average, 
                    'max' : max_val, 
                    'min' : min_val,
                    'max_time_diff': max_diff,
                    'max_date': max_date,
                    'min_date': min_date
                }
            )
    else:
        pass

results


C:\Users\jacob\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jacob\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[{'huc': 'Totuskey Creek',
  'quarter': 'Q1 2012',
  'var': 41.227788279773165,
  'stdev': 6.420886876419266,
  'avg': 15.521739130434783,
  'max': 25.0,
  'min': 2.5,
  'max_time_diff': 37,
  'max_date': Timestamp('2019-12-19 00:00:00'),
  'min_date': Timestamp('2012-01-05 00:00:00')},
 {'huc': 'Totuskey Creek',
  'quarter': 'Q1 2013',
  'var': 66.83066517013232,
  'stdev': 8.175002456888457,
  'avg': 14.190217391304348,
  'max': 30.0,
  'min': 4.5,
  'max_time_diff': 37,
  'max_date': Timestamp('2019-12-19 00:00:00'),
  'min_date': Timestamp('2012-01-05 00:00:00')},
 {'huc': 'Totuskey Creek',
  'quarter': 'Q1 2014',
  'var': 76.10523983931948,
  'stdev': 8.723831717732724,
  'avg': 15.222826086956522,
  'max': 29.5,
  'min': 1.5,
  'max_time_diff': 37,
  'max_date': Timestamp('2019-12-19 00:00:00'),
  'min_date': Timestamp('2012-01-05 00:00:00')},
 {'huc': 'Totuskey Creek',
  'quarter': 'Q1 2015',
  'var': 70.46,
  'stdev': 8.394045508573324,
  'avg': 17.033333333333335,
  'max': 32.

In [4]:
results_df = pd.DataFrame(results)
results_df.head()

,huc,quarter,var,stdev,avg,max,min,max_time_diff,max_date,min_date
0,Totuskey Creek,Q1 2012,41.227788,6.420887,15.521739,25.0,2.5,37,2019-12-19,2012-01-05
1,Totuskey Creek,Q1 2013,66.830665,8.175002,14.190217,30.0,4.5,37,2019-12-19,2012-01-05
2,Totuskey Creek,Q1 2014,76.105240,8.723832,15.222826,29.5,1.5,37,2019-12-19,2012-01-05
3,Totuskey Creek,Q1 2015,70.460000,8.394046,17.033333,32.0,3.0,37,2019-12-19,2012-01-05
4,Totuskey Creek,Q1 2016,59.024818,7.682761,16.911236,31.5,3.0,37,2019-12-19,2012-01-05
